# Operadores Genéticos

Los operadores son objetos matemáticos que toman un o más individuos y los modifican de acuerdo a lógicas que intentan emular 
el comportamiento evolutivo de las especies.

Desde aquí en adelante trabajaremos con una cantidad de skus de tamaño $Q$ y cantidad de clusters $C$. Un **individuo** es un slotting, donde su representación matricial es de la forma
$$(x_{k,i})_{(k,i)\in C\times Q}$$

Donde $x_{k,i} = 1$ si el sku $i$ está alocado en el cluster $k$, y $0$ en otro caso. 


Se recomienda la sección [Problemas](Problemas.html), para la nototación poblacional y el completo entendimiento de los operadores acá definidos.




## Sampling


Este operador se encarga de generar la población inicial, el operador de sampling es una aplicación 
$$S: N\in\mathbb{N}^{+} \rightarrow  ((x_{k,i})_{(k,i)\in C\times Q})^N \in (\mathbb{M}\{0,1\}_{C\times Q})^N$$

donde $\mathbb{M}\{0,1\}_{Q\times C}$ son las matrices de tamaño $C\times Q$ binarias.


### Problema de asignación lineal Random

Para generar un slotting random se utiliza una técnica de asignación con programación lineal. Para ello, 
se genera un matriz $\text{Cost}$ de costo aleatorio con valores enteros (con entero máximo posible como la cantidad de skus). Esta matriz es de tamaño $Q\times C$, cuya entrada $c_{k,i}$ 
representa un costo random de asignar el sku $i$ en el cluster $k$. El problema de optimización consiste en resolver

$$\max \sum_{k = 1}^{C}\sum_{i= 1}^{Q}c_{k, i}x_{k,i}$$

Sujeto a las restricciones del problema de slotting

$$
\begin{align}
  \sum_{k = 1}^{C} x_{k,i} & =  1 \hspace{0.5cm} \forall i = 1, ..., Q\\
  \sum_{i = 1}^{Q} x_{k,i} & \leq  Z_{k}  \hspace{0.5cm} \forall k = 1, ..., C\\
  \sum_{k = 1}^{C} w_{k,i}x_{k,i} & =  1 \hspace{0.5cm} \forall i = 1, ..., Q\\
  x_{k,i} & \in  \lbrace 0,1 \rbrace \\
\end{align}
$$

Las soluciones cambian radicalmente conforme cambie la matriz de costo $\text{Cost}$. La implementación de este problema lineal
está hecho en el framework de Google [ortools](https://developers.google.com/optimization/reference).


### Fast Nondominated Sorting Sampling


Esta técnica esta fundamentada en el operador Survival usado en el algoritmo ``NSGA-II``. El procedimiento es siguiente

* 1. Se fija el tamaño de la población $N$ y se genera $P = \emptyset$.

* 2. Se realiza un sampleo random generando $N$ slottings feasibles con la técnica de asignación lineal random.

* 3. A cada uno de estos slottings, se le asigna un ranking, de acuerdo a la dominancia de Pareto. Un individuo tiene ranking 0 cuando no hay ningun otro individuo que lo domine, ranking 1 si hay uno que lo domine, etc. 

* 4. Se seleccionan los $P^{\prime}$ individuos de ranking 0 (siempre hay al menos dos de ranking 0, ¿Por que?) y se hace $P = P\cup P^{\prime}$.

* 5. Si $|P| >= N$ retornar los primeros $N$ individuos, si no, seguir en el punto 2.

In [6]:
import numpy as np

from warehouse_allocation.models import ChungProblemStrictlyWeighted, ChungProblem
from warehouse_allocation.operators import NonDominatedSortingSamplingWeighted, NonDominatedSortingSampling

# Parámetros dummy
D = np.array([10,10, 10])

Z = np.array([5,5])

OCM = np.array([[0,10,10], [0,0,10], [0,0,0]])

W = np.array([5, 5, 5])

WT = np.array([[0,10], [0,10]])

# Problema con restricciones de peso
problem_weighted = ChungProblemStrictlyWeighted(D = D, Z = Z, OCM = OCM, W = W, WT = WT)

# Problema sin restricciones de pesos
problem = ChungProblem(D = D, Z = Z, OCM = OCM)

# Objeto de sampleo con retricciones de peso
sampler_weighted = NonDominatedSortingSamplingWeighted()

# Objeto de sampleo sin retricciones de peso
sampler = NonDominatedSortingSampling()

El método principal de ambas clases es ``individual``, que genéra un slotting random respetando las constraints del problema respectivo.

In [7]:
# Generamos el costo random
COST_RANDOM = sampler.random_cost(problem)
# Generamos el slotting asociado a ese costo random
sampler.individual(problem, Cost = COST_RANDOM)

array([False, False,  True,  True,  True, False])

In [8]:
# Problema con restricciones de peso
COST_RANDOM_WEIGHTED = sampler_weighted.random_cost(problem_weighted)
sampler_weighted.individual(problem_weighted, Cost = COST_RANDOM_WEIGHTED)

array([ True, False, False, False,  True,  True])

Ambas clases definen un método interno ``_do`` que es usado internamente por ``pymoo`` y se encarga de generar la población inicial ``X_pop``, llamando a los métodos exhibidos anteriormente tantas veces como sea el tamaño de la población establecido.

## Mutation

Mutar un individuo $S$ consiste en cambiar su genética mediante un proceso que involucre aleatoriedad. 
Un operador de mutación es una aplicación

$$M: (x_{k,i})_{(k,i)\in C\times Q}\in\mathbb{M}_{C\times Q} \rightarrow (x^{\prime}_{k,i})_{(k,i)\in C\times Q}\in\mathbb{M}_{C\times Q}$$


Para cada sku dado por el índice $i$, definimos el conjunto $$D(i) = A\subseteq \{1,2,...,C\}$$

que identifica cada sku con los clusters a los que puede ser alocado de acuerdo a **las restricciones de pesos**.

El operador de mutación implementado consiste en:

* Elegir un sku $i$ de forma aleatoria junto y el cluster al cual pertenece actualmente en la solución $S$, dado por el índice $k_{i}$.

* Se obtienen los candidatos a intercambio, es decir, aquellos sku $j$ tales que $k_{i}\in D(j)$. De estos, además se filtran aquellos sku $j$ tales que su cluster $k_j$ asignado en la solución sean compatibles con $i$, es decir, $k_j \in D(i)$.

* De este conjunto de candidatos final se escoge uno de forma aleatoria y se realiza un intercambio, creando así una nueva solución $S^*$.

* Se entrega la solución $S$. 

Si no hay sku posible de intercambio, es decir, esto sucede cuando un sku es exclusivo de un cluster (puede ser alocado en uno y 
solo un cluster) este operador se convierte en la identidad.

In [9]:
from warehouse_allocation.operators import ChungAislePermutationMutation, ChungAislePermutationMutationWeighted

mutation = ChungAislePermutationMutation(prob = 0.1)
mutation_weighted = ChungAislePermutationMutationWeighted(prob = 0.2)

# Individuo tiene que estar en su forma vectorial
individual = np.array([ True, False, False, False,  True,  True])

In [10]:
mutation.mutate_individual(individual, problem)

array([False, False,  True,  True,  True, False])

In [11]:
mutation_weighted.mutate_individual(individual, problem_weighted)

array([False, False,  True,  True,  True, False])

<div class="alert alert-info">
<b>Observación :</b> El parámetro <code>prob </code> es usado internamente por pymoo para aplicar el operador mutación con esa proporción a los inviduos que fueron seleccionados para el *mating*.
</div>

<div class="alert alert-info">
<b>Observación :</b> Solo se mostraron los operadores de mutación para el problema original y con restricciones de pesos, pero están definidos para el problema con división/rotación y el problema división/rotación más restricciones de pesos.
</div>


In [12]:
# Acá están :)
from warehouse_allocation.operators import ChungAislePermutationWithDivisions, ChungAislePermutationWithDivisionsPlusWeight

## Crossover


Consiste en tomar dos soluciones $S_1, S_2$ y con ellas construir una nueva solución $H$ la cual mantiene elementos 
tanto de $S_1$ como $S_2$ (Las soluciones padre). El algoritmo es como sigue:

* Se elige un pasillo $p$ de manera aleatoria.
* Se copia el estado del pasillo $p$ en la solución $S_2$ y se sobreescribe en la solución $S_1$. Esto deja un conjunto $Q^*$ de skus sin asignar. 
* Aquellos elementos que ahora están duplicados en el resto de los pasillos, se remueven. Esta acción deja a los pasillos con ciertos espacios disponibles. Se guarda esta "capacidad disponible" en un vector $Z^*$.
* Se realiza una asignación de los skus de $Q^*$ a los pasillos considerando tanto la capacidad disponible $Z^*$ como el diccionario de factibilidad $D$.
* Es posible que la asignación no tenga solución, en cuyo caso se samplea un nuevo pasillo $p^*$ y se repite la operación.

El proceso de re asignación de los SKUs se hace de manera óptima planteando un problema de flujo máximo con matriz de costo como en <cite data-cite="2020:Nadia"> Nadia </cite>. La construcción de la
matriz de costo puede ser de las siguientes dos formas

* Dado un SKU $i$ (con reindexación relativa a la reasignación) de los $Q^*$ a reasignar, el costo $c_{k,i}$ de asignar el SKU $i$ en el cluster $k$ viene dado por **afinidad/co-ocurrencia** que genera con cada uno de los SKUs ya alocados en el cluster $k$
* Dado un SKU $i$ (con reindexación relativa a la reasignación) de los $Q^*$ a reasignar, el costo $c_{k,i}$ de asignar el SKU $i$ en el cluster $k$ viene dado por la **demanda** neta tiene dicho cluster más la demanda del SKU en cuestión.

La primera matriz de costo juega a favor de la función objetivo que máximiza afinidad, y la segunda a favor de la función objetivo que regulariza
la demanda.

![Crossover](crossover.png)


La idea del crossover nace del algoritmo presentado en  <cite data-cite="1999:Larranaga"> Larranaga </cite>, pero cambia totalmente la lógica del *mapping* o reasignación.




In [13]:
from warehouse_allocation.operators import ChungPartiallyMappedCrossover, ChungPartiallyMappedCrossoverWeigthed

crossover = ChungPartiallyMappedCrossover(prob = 0.9, aff_prob = 0.5)

crossover_weighted = ChungPartiallyMappedCrossoverWeigthed(prob = 0.9, aff_prob = 0.5)

# Slotting Madre
M = np.array([False,  True, False,  True, False,  True])

# Slotting Padre
P = np.array([True, False,  True,  False,  True, False])

In [14]:
crossover.crossover(M, P, problem)

array([[False,  True, False,  True, False,  True],
       [False,  True, False,  True, False,  True]])

In [15]:
crossover_weighted.crossover(M, P, problem_weighted)

array([[ True,  True,  True, False, False, False],
       [False, False, False,  True,  True,  True]])

<div class="alert alert-info">
<b>Observación :</b> El parámetro <code>prob </code> es usado internamente por pymoo para aplicar el operador 
    crossover con esa proporción a los inviduos que fueron seleccionados para el *mating*.
    Además se recibe como parámetro <code> aff_prob </code>, que indica la probabilidad de jugar a 
    favor de la afinidad en la reasignación/mapping descrito anteriormente.

</div>

<div class="alert alert-info">
<b>Observación :</b> Solo se mostraron los operadores de crossover para el problema original y con restricciones de pesos, pero están definidos para el problema con división/rotación y el problema división/rotación más restricciones de pesos.
</div>


In [17]:
# Acá está :)
from warehouse_allocation.operators import ChungPartiallyMappedCrossoverWithDivisions

# Esta clase es usada tanto para el problema con divisiones/rotaciones y el problema con divisiones/rotaciones + 
# restricciones de peso, ver la API doc para ver el uso